In [ ]:
from google.colab import drive
drive.mount('/content/drive/',force_remount=True)

Mounted at /content/drive/


In [ ]:
!pip install tensorflow numpy pandas tqdm scikit-learn segmentation-models tensorflow-addons

In [ ]:
!pip install kaggle

In [ ]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"alondraaraya","key":"5a6c33331926be9146c57ad615e1f32c"}'}

In [ ]:
!mkdir -p ~/.kaggle
!mv kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle datasets download -d gomezp/benchmark-for-automatic-glottis-segmentation

100% 16.3G/16.4G [02:19<00:00, 79.7MB/s]
100% 16.4G/16.4G [02:19<00:00, 126MB/s] 


In [ ]:
!unzip benchmark-for-automatic-glottis-segmentation.zip


In [ ]:
import tensorflow as tf
import os
os.environ['SM_FRAMEWORK'] = 'tf.keras'
import numpy as np
import pandas as pd
import random
from tqdm.notebook import tqdm

from sklearn.model_selection import train_test_split
from segmentation_models.losses import dice_loss
from segmentation_models.metrics import iou_score

from drive.MyDrive.Glottis.colab_glottis.GlottisNetV2.Utils.DataGenerator import DataGenerator
from drive.MyDrive.Glottis.colab_glottis.GlottisNetV2.Utils.data import load_data, metric_mape, mape_ap, mape_pp
from drive.MyDrive.Glottis.colab_glottis.GlottisNetV2.Utils.Callbacks import get_callbacks
from drive.MyDrive.Glottis.colab_glottis.GlottisNetV2.Models.GlottisNetV2_e import glottisnetV2_e

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

Segmentation Models: using `tf.keras` framework.


/usr/local/lib/python3.10/dist-packages/tensorflow_addons/utils/tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(


In [ ]:
# Set path to the text file with coordinates of the anterior and posterior points
coord_train = r"drive/MyDrive/Glottis/points/training/ap.points" #TODO

# Set path to training data
img_training = r"/content/training/training" # TODO

N_train = 55750 # number of training images

# Create video IDs for training data and save them in Pandas Dataframe
cols = ['z','path']
df_imgs_train = pd.DataFrame(columns= cols)
df_segs_train = pd.DataFrame(columns =cols)

for i in tqdm(range(N_train)):
    row_imgs = {'z' : [i], 'path': [img_training + "/" + str(i) + ".png"]}
    row_segs = {'z': [i], 'path': [img_training + "/" + str(i) + '_seg.png']}
    df_imgs_train = pd.concat([df_imgs_train, pd.DataFrame(row_imgs)])
    df_segs_train = pd.concat([df_segs_train, pd.DataFrame(row_segs)])

print('Created IDs for training images.')

# Save coordinates of anterior and posterior points in Pandas Dataframe
training_data = load_data(coord_train, N_train)
print('Loaded anterior and posterior points to dataframe.')

  0%|          | 0/55750 [00:00<?, ?it/s]

Created IDs for training images.


  0%|          | 0/111498 [00:00<?, ?it/s]

One coordinate is missing in image:  11170
One coordinate is missing in image:  43704
Loaded anterior and posterior points to dataframe.


In [ ]:
'''Training'''
# Set random seed for reproducible training
SEED = 42
os.environ['PYTHONHASHSEED'] = str(SEED)
rand=np.random.seed(SEED)
random.seed(SEED)
tf.compat.v1.set_random_seed(SEED)

# Set parameters
BATCH_SIZE = 8
FILTERS = 16
LAYERS= 4
LEARNING_RATE = 0.2e-3
EPOCHS = 30
TARGET_HEIGHT = 512
TARGET_WIDTH = 256
SHUFFLE = True
AUGMENT = True
MODEL_PATH = r"drive/MyDrive/Glottis/Model/model.h5"
STEPS_PATH = r"drive/MyDrive/Glottis/Model/steps"
N_STEPS = 5 # Save every #N_STEPS epoch
RADIUS = 15

model = glottisnetV2_e(input_size=(TARGET_HEIGHT, TARGET_WIDTH, 1), layers=LAYERS, filters=FILTERS)

# Hard split of training and validation data
train_imgs, val_imgs, train_segs, val_segs = train_test_split(df_imgs_train,
                                                              df_segs_train,
                                                              test_size = 0.1,
                                                              random_state = SEED)

# Training data --> Augmentation and Shuffle
training_generator = DataGenerator(train_imgs, train_segs, batch_size = BATCH_SIZE, target_height = TARGET_HEIGHT, \
                                   target_width = TARGET_WIDTH, shuffle = SHUFFLE, df_coordinates = training_data, \
                                   augment = AUGMENT, radius=RADIUS)

# Validation data
validation_generator = DataGenerator(val_imgs, val_segs, target_height = TARGET_HEIGHT, \
                                     target_width = TARGET_WIDTH, batch_size = BATCH_SIZE, shuffle = False, \
                                     df_coordinates = training_data, augment = False, radius=RADIUS)

# Compile model with dice_loss for segmentation, mse for prediction maps and use Adam as optimizer
# First exit: predictions of anterior and posterior points (2 channels)
# Second exit: Segmentations
model.compile(optimizer = tf.keras.optimizers.Adam(learning_rate = LEARNING_RATE), \
              metrics = {'seg': ['acc', iou_score],
                         'ap_pred': ['acc', metric_mape, mape_ap, mape_pp]},
              loss = {'ap_pred': 'mse', 'seg': dice_loss}, run_eagerly=True)

# Train model on dataset and save it
model.fit(training_generator, validation_data= validation_generator, epochs = EPOCHS,
                    callbacks = get_callbacks(MODEL_PATH, model, N_STEPS, STEPS_PATH))



Epoch 1/30


6271/6271 [==============================] - ETA: 0s - loss: 0.3332 - ap_pred_loss: 0.0136 - seg_loss: 0.3196 - ap_pred_acc: 0.9850 - ap_pred_metric_mape: 36.8902 - ap_pred_mape_ap: 44.6855 - ap_pred_mape_pp: 29.1952 - seg_acc: 0.9921 - seg_iou_score: 0.5798

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


6271/6271 [==============================] - 4569s 724ms/step - loss: 0.3332 - ap_pred_loss: 0.0136 - seg_loss: 0.3196 - ap_pred_acc: 0.9850 - ap_pred_metric_mape: 36.8902 - ap_pred_mape_ap: 44.6855 - ap_pred_mape_pp: 29.1952 - seg_acc: 0.9921 - seg_iou_score: 0.5798 - val_loss: 0.0980 - val_ap_pred_loss: 0.0017 - val_seg_loss: 0.0963 - val_ap_pred_acc: 0.9769 - val_ap_pred_metric_mape: 22.1737 - val_ap_pred_mape_ap: 25.0707 - val_ap_pred_mape_pp: 19.2767 - val_seg_acc: 0.9986 - val_seg_iou_score: 0.8300
Epoch 2/30
6271/6271 [==============================] - ETA: 0s - loss: 0.1192 - ap_pred_loss: 0.0017 - seg_loss: 0.1176 - ap_pred_acc: 0.9538 - ap_pred_metric_mape: 23.3349 - ap_pred_mape_ap: 30.1503 - ap_pred_mape_pp: 16.5837 - seg_acc: 0.9983 - seg_iou_score: 0.7969

6271/6271 [==============================] - 4433s 707ms/step - loss: 0.1192 - ap_pred_loss: 0.0017 - seg_loss: 0.1176 - ap_pred_acc: 0.9538 - ap_pred_metric_mape: 23.3349 - ap_pred_mape_ap: 30.1503 - ap_pred_mape_pp: 16.5837 - seg_acc: 0.9983 - seg_iou_score: 0.7969 - val_loss: 0.0897 - val_ap_pred_loss: 0.0014 - val_seg_loss: 0.0883 - val_ap_pred_acc: 0.9176 - val_ap_pred_metric_mape: 9.6826 - val_ap_pred_mape_ap: 11.1050 - val_ap_pred_mape_pp: 8.2602 - val_seg_acc: 0.9987 - val_seg_iou_score: 0.8425
Epoch 3/30
1043/6271 [===>..........................] - ETA: 58:32 - loss: 0.1110 - ap_pred_loss: 0.0015 - seg_loss: 0.1095 - ap_pred_acc: 0.8836 - ap_pred_metric_mape: 13.8769 - ap_pred_mape_ap: 17.6696 - ap_pred_mape_pp: 10.1489 - seg_acc: 0.9984 - seg_iou_score: 0.8082

FailedPreconditionError: ignored